# Data Cleaning Listado de Listado de Beneficiarios 2021

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_21 -> Dataset con el listado de TODOS los beneficiarios de 2021.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2021

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_21 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2021.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_21.shape

(394986, 16)

In [5]:
beneficiarios_21 = beneficiarios_21[~((beneficiarios_21['ENTIDAD'] == 'NACIONAL') & (beneficiarios_21['MUNICIPIO'] == 'NACIONAL') & (beneficiarios_21['LOCALIDAD'] == 'NACIONAL'))]

In [6]:
beneficiarios_21['ENTIDAD'].unique()

array(['GUERRERO', 'PUEBLA', 'MORELOS', 'TLAXCALA', nan], dtype=object)

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [7]:
beneficiarios_21.dropna(inplace=True)

In [8]:
# Obtenemos las localidades únicas en el dataset.
Localidades_21 = beneficiarios_21[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_21 = Localidades_21.drop_duplicates()


In [9]:
Localidades_21['ENTIDAD_c_benef'] = Localidades_21['ENTIDAD'].apply(clean_text)
Localidades_21['MUNICIPIO_c_benef'] = Localidades_21['MUNICIPIO'].apply(clean_text)
Localidades_21['LOCALIDAD_c_benef'] = Localidades_21['LOCALIDAD'].apply(clean_text)

In [10]:
# Creamos las dos keys de beneficiarios 2019
Localidades_21['KEY_benef_mun'] = Localidades_21['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_21[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_21['KEY_benef_loc'] = Localidades_21['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_21[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_21['LOCALIDAD_c_benef'].astype(str)

In [11]:
Localidades_21.shape

(5914, 8)

In [12]:
# Obtenemos las localidades únicas en el dataset.
Municipios_21 = beneficiarios_21[['ENTIDAD', 'MUNICIPIO']]
Municipios_21 = Municipios_21.drop_duplicates()

In [13]:
# Estandarizamos la limpieza de los datos
Municipios_21['ENTIDAD_c_benef'] = Municipios_21['ENTIDAD'].apply(clean_text)
Municipios_21['MUNICIPIO_c_benef'] = Municipios_21['MUNICIPIO'].apply(clean_text)

In [14]:
# Creamos las dos keys de beneficiarios 
Municipios_21['KEY_benef_mun'] = Municipios_21['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_21[
    'MUNICIPIO_c_benef'].astype(str)

In [15]:
Municipios_21.shape

(248, 5)

In [16]:
Municipios_21.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2021

Obtener listado único de municipios y listado único de localidades de inegi de 2021 con sus claves.

In [17]:
path_dataset_inegi_2021 = '../../data/inegi/dataset_inegi_clean_2021.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2021 = pd.read_csv(path_dataset_inegi_2021)

### 1.2.1 INEGI 2021 Municipios únicos para cada año.

In [18]:
dataset_inegi_2021['KEY_inegi_municipio'] = dataset_inegi_2021['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2021[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2021['KEY_inegi_localidad'] = dataset_inegi_2021['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2021[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2021['Localidad_c_inegi'].astype(str)

In [19]:
INEGI_UNIQUEMUN_2021 = dataset_inegi_2021.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad", "POB_TOTAL"])

INEGI_UNIQUEMUN_2021 = INEGI_UNIQUEMUN_2021.drop_duplicates()

In [20]:
INEGI_UNIQUEMUN_2021.shape

(2471, 7)

In [21]:
INEGI_UNIQUEMUN_2021.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2021

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2021, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_21 y el dataset de INEGI correspondiente.

In [22]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left')
    
    return df_merged

In [23]:
diccionario_MUN_21 = fuzzy_merge_benef2019_2022(Municipios_21, INEGI_UNIQUEMUN_2021, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_21.shape

(248, 14)

In [24]:
diccionario_MUN_21.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)

In [25]:
diccionario_MUN_21.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_21.csv', index=False)

In [26]:
diccionario_MUN_21_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_21_simple.csv')

In [27]:
nan_rows = beneficiarios_21[beneficiarios_21.isna().any(axis=1)]

In [28]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_21[beneficiarios_21['ENTIDAD'] == 'NACIONAL']

### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [29]:
beneficiarios_21.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [30]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_21['ESTADO_Clean'] = beneficiarios_21['ENTIDAD'].apply(clean_text)
beneficiarios_21['MUNICIPIO_Clean'] = beneficiarios_21['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_21['Estado-mun-KEY'] = beneficiarios_21['ESTADO_Clean'].astype(str) + '-' + beneficiarios_21[
    'MUNICIPIO_Clean'].astype(str)

In [31]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_21, diccionario_MUN_21_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_mun", how='left', suffixes=('_benef', '_inegi'))

In [32]:
listado_beneficiarios_parte_I.isna().sum()

BENEFICIARIO           0
ZONA                   0
ENTIDAD                0
MUNICIPIO              0
LOCALIDAD              0
ESTRATIFICACIÓN        0
PROGRAMA               0
COMPONENTE             0
SUBCOMPONENTE          0
PRODUCTO               0
FECHA                  0
MONTO FEDERAL          0
APOYO                  0
ACTIVIDAD              0
ESLABÓN                0
CICLO AGRÍCOLA         0
ESTADO_Clean           0
MUNICIPIO_Clean        0
Estado-mun-KEY         0
KEY_benef_mun          0
KEY_inegi_municipio    0
dtype: int64

In [33]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [34]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2021, left_on="KEY_inegi_municipio",
                                        right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [35]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'Entidad_c_inegi','Municipio_c_inegi', 'KEY_inegi_municipio', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [36]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [37]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2021.csv', index=False)

In [38]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
KEY_benef_mun      0
CVE_ENT            0
Entidad_inegi      0
CVE_MUN            0
Municipio_inegi    0
dtype: int64

In [39]:
listado_beneficiarios_parte_II.shape

(394983, 14)

# 3. Join por entidad (Divide y Vencerás)

In [40]:
listado_beneficiarios_2021 = pd.read_csv('../../data/listados_completos/listado_beneficiarios_2021.csv')

In [41]:
listado_beneficiarios_2021.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [42]:
listado_beneficiarios_2021['ENTIDAD_c_benef'] = listado_beneficiarios_2021['ENTIDAD'].apply(clean_text)
listado_beneficiarios_2021['MUNICIPIO_c_benef'] = listado_beneficiarios_2021['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_2021['LOCALIDAD_c_benef'] = listado_beneficiarios_2021['LOCALIDAD'].apply(clean_text)

In [43]:
listado_beneficiarios_2021['KEY_benef_loc'] = listado_beneficiarios_2021['MUNICIPIO_c_benef'].astype(str) + '-' + listado_beneficiarios_2021['LOCALIDAD_c_benef'].astype(str)

In [44]:
listado_beneficiarios_2021.shape

(394983, 18)

In [45]:
listado_beneficiarios_2021 = listado_beneficiarios_2021.drop_duplicates(subset=['KEY_benef_loc'])

In [46]:
listado_beneficiarios_2021.shape

(5913, 18)

In [47]:
listado_beneficiarios_2021.ENTIDAD.unique()

array(['GUERRERO', 'PUEBLA', 'MORELOS', 'TLAXCALA'], dtype=object)

### 3.2.1 Join de localidades - GUERRERO

Empezamos creando los datasets por entiedad del dataset de beneficiarios.


In [48]:
entities = ['GUERRERO', 'PUEBLA', 'MORELOS', 'TLAXCALA']

# Dictionary to hold DataFrames
dataframes_dict = {}

for entity in entities:
    variable_name = f"Localidades_21_{entity.upper().replace(' ', '_')}"
    dataframe = listado_beneficiarios_2021[listado_beneficiarios_2021["ENTIDAD"] == entity]
    
    # Check if the DataFrame for this entity already exists in the dictionary
    if variable_name in dataframes_dict:
        # If it exists, concatenate the new DataFrame with the existing one
        dataframes_dict[variable_name] = pd.concat([dataframes_dict[variable_name], dataframe])
    else:
        # If it does not exist, create a new entry in the dictionary
        dataframes_dict[variable_name] = dataframe

# Define each DataFrame as a separate variable
for name, df in dataframes_dict.items():
    globals()[name] = df
    print(f"{name} defined with shape: {df.shape}")

Localidades_21_GUERRERO defined with shape: (4588, 18)
Localidades_21_PUEBLA defined with shape: (646, 18)
Localidades_21_MORELOS defined with shape: (273, 18)
Localidades_21_TLAXCALA defined with shape: (406, 18)


In [49]:
listado_beneficiarios_2021.shape

(5913, 18)

### 3.2.2 Diccionarios de Inegi por entidad.

In [50]:
folder_path = '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/2021'

# Listar todos los archivos en la carpeta
files = os.listdir(folder_path)

# Filtrar solo los archivos CSV
csv_files = [file for file in files if file.endswith('.csv')]

# Leer cada archivo CSV y almacenar en variables separadas
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    # Crear un nombre de variable dinámicamente eliminando la extensión .csv
    var_name = file_name.replace('.csv', '')
    # Leer el archivo CSV y asignar el DataFrame a la variable
    globals()[var_name] = pd.read_csv(file_path)

# Opcional: Imprimir el nombre de las variables creadas y la cantidad de filas y columnas de cada DataFrame
for file_name in csv_files:
    var_name = file_name.replace('.csv', '')
    df = globals()[var_name]
    print(f'{var_name}: {df.shape[0]} filas, {df.shape[1]} columnas')


INEGI_UNIQUELOC_2021_morelos: 1905 filas, 11 columnas
INEGI_UNIQUELOC_2021_guerrero: 9214 filas, 11 columnas
INEGI_UNIQUELOC_2021_puebla: 7724 filas, 11 columnas
INEGI_UNIQUELOC_2021_tlaxcala: 1685 filas, 11 columnas


In [51]:
INEGI_UNIQUELOC_2021_guerrero.shape

(9214, 11)

In [52]:
Localidades_21_GUERRERO_parte_I = fuzzy_merge_benef2019_2022(Localidades_21_GUERRERO, INEGI_UNIQUELOC_2021_guerrero, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_21_GUERRERO_parte_I.shape

Localidades_21_PUEBLA_parte_I = fuzzy_merge_benef2019_2022(Localidades_21_PUEBLA, INEGI_UNIQUELOC_2021_puebla, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_21_PUEBLA_parte_I.shape

Localidades_21_MORELOS_parte_I = fuzzy_merge_benef2019_2022(Localidades_21_MORELOS, INEGI_UNIQUELOC_2021_morelos, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_21_MORELOS_parte_I.shape

Localidades_21_TLAXCALA_parte_I = fuzzy_merge_benef2019_2022(Localidades_21_TLAXCALA, INEGI_UNIQUELOC_2021_tlaxcala, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_21_TLAXCALA_parte_I.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_18940/3519495578.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_18940/3519495578.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_18940/3519495578.py:18: SettingWithCopyWar

(406, 31)

In [53]:
# Concatenate datasets vertically
diccionario_LOC_21_simple = pd.concat([
    Localidades_21_GUERRERO_parte_I,
    Localidades_21_PUEBLA_parte_I,
    Localidades_21_MORELOS_parte_I,
    Localidades_21_TLAXCALA_parte_I
], axis=0)

# Check the shape of the concatenated DataFrame
diccionario_LOC_21_simple.shape

(5915, 31)

In [54]:
diccionario_LOC_21_simple.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun',
       'CVE_ENT_x', 'Entidad_inegi_x', 'CVE_MUN_x', 'Municipio_inegi_x',
       'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef',
       'KEY_benef_loc', 'best_match', 'match_score', 'CVE_ENT_y',
       'Entidad_inegi_y', 'CVE_MUN_y', 'Municipio_inegi_y', 'CVE_LOC',
       'Localidad_inegi', 'POB_TOTAL', 'Entidad_c_inegi', 'Municipio_c_inegi',
       'Localidad_c_inegi', 'KEY_inegi_localidad'],
      dtype='object')

In [55]:
diccionario_LOC_21_simple.drop(columns=['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'ENTIDAD_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'best_match', 'CVE_ENT_x', 'Entidad_inegi_x', 'CVE_MUN_x', 'Municipio_inegi_x',
       'CVE_LOC', 'Localidad_inegi', 'POB_TOTAL', 'Entidad_c_inegi',
       'Municipio_c_inegi', 'Localidad_c_inegi', 'CVE_ENT_y', 'Entidad_inegi_y', 'CVE_MUN_y', 'Municipio_inegi_y', 'KEY_benef_mun'], inplace=True)

In [56]:
diccionario_LOC_21_simple.columns

Index(['KEY_benef_loc', 'match_score', 'KEY_inegi_localidad'], dtype='object')

In [59]:
diccionario_LOC_21_simple.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_21_simple.csv', index=False)

In [60]:
diccionario_LOC_21_simple.isna().sum()

KEY_benef_loc          0
match_score            0
KEY_inegi_localidad    0
dtype: int64